In [1]:
import os
import pandas as pd
import asyncio
from translator import translate_column_to_english
from local_matching import load_and_match, evaluate_matching, clean_file
from langchain_matcher import process_all_batches

# Get the absolute path of the parent directory (one level up)
script_dir = os.getcwd()  # Jupyter notebooks run in the current working directory
parent_dir = os.path.abspath(os.path.join(script_dir, os.pardir))  # Move one folder up

# Construct paths to parallel folders
data_dir = os.path.join(parent_dir, "data")
softwaregini_dir = os.path.join(parent_dir, "softwaregini_files")

# Ensure the directory exists
os.makedirs(data_dir, exist_ok=True)
os.makedirs(softwaregini_dir, exist_ok=True)

# Define file paths
output_file = os.path.join(data_dir, "full_matched_results.csv")
ground_truth_file = os.path.join(data_dir, "ground_truth.csv")
input_file = os.path.join(softwaregini_dir, "organization_tools.csv")

# Run translation (handle async functions in Jupyter properly)
df = await translate_column_to_english(input_file)



Translating 327 texts asynchronously...
Translation complete.


In [2]:
df = clean_file(df)

Cleaning file...


In [3]:
# Load the translated data and match locally

potential_matches = []
for i in range(75, 81, 5):
    df_matched, potential_matches = load_and_match(df, similarity_cutoff=i)
    # Evaluate results
    results = evaluate_matching(df_matched, ground_truth_file)
    print(f"Results before LLM for cutoff {i}: ", results)

    # Ensure column names are stripped of any leading/trailing spaces
    potential_matches.columns = potential_matches.columns.str.strip()

        # Now safely select and rename
    potential_matches = potential_matches[['human_name', 'human_description']].rename(
            columns={'human_name': 'name', 'human_description': 'description'}
    )
    # Process matches asynchronously using to_thread (since process_all_batches is not async)
    df_potential = process_all_batches(potential_matches, 95, 50)

    # Merge results
    df_final = pd.concat([df_matched, df_potential], axis=0)

    results = evaluate_matching(df_final, ground_truth_file)
    print(f"Results with LLM For cutoff {i}:", results) 



# Debug: Print columns to check data
#print("Columns in potential_matches:", potential_matches.columns)


Computing embeddings for human descriptions...
Matching human descriptions to tools...
Amount of total Matches:  152
Amount of potential Matches:  5634
Number of dropped/deleted matches: 34
Number of final matches: 118
Number of potential matches: 335
Precision: 0.958
Recall: 0.706
F1-Score: 0.813
True Positives: 113
False Positives: 5
False Negatives: 47
False Positives List:
False Negatives List:
Results before LLM for cutoff 75:  {'Precision': 0.9576271186440678, 'Recall': 0.70625, 'F1-Score': 0.8129496402877698, 'True Positives': 113, 'False Positives': 5, 'False Negatives': 47, 'False Positives List': [('microsoft teams', 'microsoft teams room'), ('crystal reports activex viewer', 'sap crystal'), ('crystal report runtime sp6', 'sap crystal'), ('mindjet mindmanager reader', 'mindmanager'), ('libreoffice', 'libreoffice')], 'False Negatives List': [('skype meetings', 'skype'), ('mikogo', 'beamyourscreen'), ('sendinblue', 'brevo transactional email'), ('anaconda3', 'anaconda'), ('dyna

100%|██████████| 7/7 [03:02<00:00, 26.09s/it]


Precision: 0.950
Recall: 0.825
F1-Score: 0.883
True Positives: 132
False Positives: 7
False Negatives: 28
False Positives List:
False Negatives List:
Results with LLM For cutoff 75: {'Precision': 0.9496402877697842, 'Recall': 0.825, 'F1-Score': 0.882943143812709, 'True Positives': 132, 'False Positives': 7, 'False Negatives': 28, 'False Positives List': [('inkscape', 'camtasia'), ('textpad', 'textpad'), ('microsoft teams', 'microsoft teams room'), ('crystal reports activex viewer', 'sap crystal'), ('crystal report runtime sp6', 'sap crystal'), ('mindjet mindmanager reader', 'mindmanager'), ('libreoffice', 'libreoffice')], 'False Negatives List': [('mikogo', 'beamyourscreen'), ('sendinblue', 'brevo transactional email'), ('anaconda3', 'anaconda'), ('dynamincs nav app', 'microsoft dynamics 365 business central'), ('smartanalyzer app sdk', 'caseware idea'), ('sendinblue', 'brevo marketing platform'), ('abbyy flexicapture', 'abbyy flexicapture for invoices'), ('microsoft office', 'microsof

100%|██████████| 7/7 [02:58<00:00, 25.45s/it]

Precision: 0.940
Recall: 0.781
F1-Score: 0.853
True Positives: 125
False Positives: 8
False Negatives: 35
False Positives List:
False Negatives List:
Results with LLM For cutoff 80: {'Precision': 0.9398496240601504, 'Recall': 0.78125, 'F1-Score': 0.8532423208191127, 'True Positives': 125, 'False Positives': 8, 'False Negatives': 35, 'False Positives List': [('inkscape', 'camtasia'), ('gotomeeting', 'join.me'), ('textpad', 'textpad'), ('microsoft power bi report builder', 'microsoft® report builder'), ('microsoft teams', 'microsoft teams room'), ('mindjet mindmanager reader', 'mindmanager'), ('ringcentral meetings', 'anywhere conferencing'), ('libreoffice', 'libreoffice')], 'False Negatives List': [('mikogo', 'beamyourscreen'), ('sendinblue', 'brevo transactional email'), ('anaconda3', 'anaconda'), ('dynamincs nav app', 'microsoft dynamics 365 business central'), ('smartanalyzer app sdk', 'caseware idea'), ('github desktop', 'github'), ('sendinblue', 'brevo marketing platform'), ('abbyy

In [4]:
# Evaluate results
results = evaluate_matching(df_matched, ground_truth_file)
print(results)

Precision: 0.980
Recall: 0.606
F1-Score: 0.749
True Positives: 97
False Positives: 2
False Negatives: 63
False Positives List:
False Negatives List:
{'Precision': 0.9797979797979798, 'Recall': 0.60625, 'F1-Score': 0.7490347490347489, 'True Positives': 97, 'False Positives': 2, 'False Negatives': 63, 'False Positives List': [('microsoft teams', 'microsoft teams room'), ('mindjet mindmanager reader', 'mindmanager')], 'False Negatives List': [('skype meetings', 'skype'), ('mikogo', 'beamyourscreen'), ('sendinblue', 'brevo transactional email'), ('anaconda3', 'anaconda'), ('dynamincs nav app', 'microsoft dynamics 365 business central'), ('barracuda network access client', 'barracuda network access client'), ('smartanalyzer app sdk', 'caseware idea'), ('github desktop', 'github'), ('sendinblue', 'brevo marketing platform'), ('abbyy flexicapture', 'abbyy flexicapture for invoices'), ('microsoft office', 'microsoft 365'), ('gotomeeting', 'goto meeting'), ('microsoft visio viewer', 'visio'), (

In [5]:
# Debug: Print columns to check data
print("Columns in df:", df.columns)
print([col for col in potential_matches.columns])  # Should print clean column names

# Ensure column names are stripped of any leading/trailing spaces
potential_matches.columns = potential_matches.columns.str.strip()

    # Now safely select and rename
potential_matches = potential_matches[['human_name', 'human_description']].rename(
        columns={'human_name': 'name', 'human_description': 'description'}
)
print([col for col in potential_matches.columns])

Columns in df: Index(['id', 'organization_id', 'name', 'description', 'in_house',
       'embedding'],
      dtype='object')
['name', 'description']


KeyError: "None of [Index(['human_name', 'human_description'], dtype='object')] are in the [columns]"

In [37]:
# Process matches asynchronously using to_thread (since process_all_batches is not async)
df_potential = process_all_batches(potential_matches, 98, 50)

# Merge results
df_final = pd.concat([df, df_potential], axis=0)

100%|██████████| 3/3 [01:10<00:00, 23.64s/it]


In [38]:
# Evaluate results
results = evaluate_matching(df_final, ground_truth_file)
print(results)

Precision: 0.981
Recall: 0.637
F1-Score: 0.773
True Positives: 102
False Positives: 2
False Negatives: 58
False Positives List:
False Negatives List:
{'Precision': 0.9807692307692307, 'Recall': 0.6375, 'F1-Score': 0.7727272727272727, 'True Positives': 102, 'False Positives': 2, 'False Negatives': 58, 'False Positives List': [('libreoffice', 'libreoffice'), ('mindjet mindmanager reader', 'mindmanager')], 'False Negatives List': [('teamdrive', 'synqion'), ('abbyy flexicapture', 'abbyy flexicapture for invoices'), ('articulate studio', 'articulate 360'), ('cosinex bietertool', 'cosinex vergabemarktplatz (vmp)'), ('icloud', 'apple icloud'), ('pvsol premium', 'pv*sol'), ('pafnow companion', 'pafnow process mining'), ('zoom', 'zoom'), ('sparkol videoscribe', 'videoscribe'), ('microsoft project standard', 'microsoft project'), ('gotoassist expert', 'goto resolve (formerly gotoassist)'), ('reflectorapp', 'reflector'), ('vitero starter', 'vitero inspire'), ('bkms incident reporting', 'bkms® sys

In [2]:
from langchain_matcher import process_all_batches

input_file_2 = os.path.join(softwaregini_dir, "organization_tools_2.csv")
ground_truth_file_2 = os.path.join(data_dir, "ground_truth_2.csv")

# Run translation (handle async functions in Jupyter properly)
df_2 = await translate_column_to_english(input_file_2)
df_2 = clean_file(df_2)

df_matched_2, potential_matches_2 = load_and_match(df_2, similarity_cutoff=75)
results_2 = evaluate_matching(df_matched_2, ground_truth_file_2)
print(f"Results before LLM for cutoff 75: ", results_2)

# Ensure column names are stripped of any leading/trailing spaces
potential_matches_2.columns = potential_matches_2.columns.str.strip()

# Now safely select and rename
potential_matches_2 = potential_matches_2[['human_name', 'human_description']].rename(
    columns={'human_name': 'name', 'human_description': 'description'})
# Process matches asynchronously using to_thread (since process_all_batches is not async)
df_llm_2, df_recommended = process_all_batches(potential_matches_2, 95, 50)

# Merge results
df_final_2 = pd.concat([df_matched_2, df_llm_2], axis=0)
#df_final_2 = clean_file(df_final_2)
results_2 = evaluate_matching(df_final_2, ground_truth_file_2, input_file_2)
print(f"Recommended tools by AI:", df_recommended)

print(df_recommended)

Translating 110 texts asynchronously...
Translation complete.
Cleaning file...
Computing embeddings for human descriptions...
Matching human descriptions to tools...
Amount of total Matches:  119
Amount of potential Matches:  3144
Number of dropped/deleted matches: 37
Number of final matches: 82
Number of potential matches: 145
Precision: 0.988
Recall: 0.862
F1-Score: 0.920
True Positives: 81
False Positives: 1
False Negatives: 13
False Positives List:
False Negatives List:
Results before LLM for cutoff 75:  {'Precision': 0.9878048780487805, 'Recall': 0.8617021276595744, 'F1-Score': 0.9204545454545454, 'True Positives': 81, 'False Positives': 1, 'False Negatives': 13, 'False Positives List': [('linkedin', 'linkedin learning')], 'False Negatives List': [('sap leanix application portfolio management', 'leanix saas management platform'), ('sap leanix application portfolio management', 'leanix enterprise architecture management'), ('bic enterprise risk', 'bic grc'), ('knowbe4', 'knowbe4 se

100%|██████████| 3/3 [01:22<00:00, 27.37s/it]

<class 'pandas.core.frame.DataFrame'>                                  human_name                     tool_name  \
48                                     miro                          miro   
73                                 docusign                      docusign   
71                       microsoft entra id            microsoft entra id   
70                       microsoft entra id            microsoft entra id   
69                  bitdefender gravityzone       bitdefender gravityzone   
..                                      ...                           ...   
38                     atlassian confluence                    confluence   
52                       atlassian opsgenie                      opsgenie   
39                               teamviewer             teamviewer remote   
41  codetwo email signatures for office 365  codetwo email signatures 365   
0                   microsoft visual studio                 visual studio   

    similarity_score  
48            

## Async Part

In [5]:
import os
import pandas as pd
from langchain_matcher_async import process_all_batches_async
import nest_asyncio
from translator import translate_column_to_english
from local_matching import load_and_match, evaluate_matching, clean_file

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Define file paths
softwaregini_dir = "../softwaregini_files"
data_dir = "../data"
input_file = os.path.join(softwaregini_dir, "organization_tools.csv")
ground_truth_file = os.path.join(data_dir, "ground_truth.csv")

# Load and preprocess data
df_async = await translate_column_to_english(input_file)  # Assuming this is an async function
df_async = clean_file(df_async)  # Assuming this is a synchronous function

# Perform matching and evaluation
df_matched_async, potential_matches_async = load_and_match(df_async, similarity_cutoff=75)  # Assuming this is a synchronous function
results_async = evaluate_matching(df_matched_async, ground_truth_file, input_file)  # Assuming this is a synchronous function
print(f"Results before LLM for cutoff 75: ", results_async)

# Clean and prepare potential matches for LLM processing
potential_matches_async.columns = potential_matches_async.columns.str.strip()
potential_matches_async = potential_matches_async[['human_name', 'human_description']].rename(
    columns={'human_name': 'name', 'human_description': 'description'})

# Process potential matches asynchronously
df_potential_async , df_recommended_async = await process_all_batches_async(potential_matches_async, 95, 50)

# Merge results and evaluate
df_final_async = pd.concat([df_matched_async, df_recommended_async], axis=0)
df_final_async = clean_file(df_final_async)
results_async = evaluate_matching(df_final_async, ground_truth_file, input_file)
print(f"Results after LLM: ", results_async)

Translating 327 texts asynchronously...
Translation complete.
Cleaning file...
Computing embeddings for human descriptions...
Matching human descriptions to tools...
Amount of total Matches:  154
Amount of potential Matches:  5614
Number of dropped/deleted matches: 35
Number of final matches: 119
Number of potential matches: 333
Precision: 0.958
Recall: 0.713
F1-Score: 0.817
True Positives: 114
True Negatives: 162
False Positives: 5
False Negatives: 46
False Positives List:
False Negatives List:
Results before LLM for cutoff 75:  {'Precision': 0.957983193277311, 'Recall': 0.7125, 'F1-Score': 0.8172043010752688, 'True Positives': 114, 'True Negatives': 162, 'False Positives': 5, 'False Negatives': 46, 'False Positives List': [('mindjet mindmanager reader', 'mindmanager'), ('libreoffice', 'libreoffice'), ('microsoft teams', 'microsoft teams room'), ('crystal report runtime sp6', 'sap crystal'), ('crystal reports activex viewer', 'sap crystal')], 'False Negatives List': [('bkms incident r

100%|██████████| 7/7 [00:24<00:00,  3.44s/it]

Cleaning file...
Precision: 0.960
Recall: 0.750
F1-Score: 0.842
True Positives: 120
True Negatives: 162
False Positives: 5
False Negatives: 40
False Positives List:
False Negatives List:
Results after LLM:  {'Precision': 0.96, 'Recall': 0.75, 'F1-Score': 0.8421052631578947, 'True Positives': 120, 'True Negatives': 162, 'False Positives': 5, 'False Negatives': 40, 'False Positives List': [('mindjet mindmanager reader', 'mindmanager'), ('libreoffice', 'libreoffice'), ('microsoft teams', 'microsoft teams room'), ('crystal report runtime sp6', 'sap crystal'), ('crystal reports activex viewer', 'sap crystal')], 'False Negatives List': [('bkms incident reporting', 'bkms® system'), ('dpma direkt', 'dpmadirektpro'), ('sophos ssl vpn client', 'sophos firewall'), ('teamdrive', 'synqion'), ('microsoft office', 'microsoft 365'), ('google chrome', 'chrome'), ('pafnow companion', 'pafnow process mining'), ('bildungsinnovator lernkarten', 'lxt authoring'), ('sendinblue', 'brevo marketing platform'), 

#### Pipeline for Organization_tools_2: Translation -> Cleaning -> Local Matching -> LLM Matching -> Evaluation

In [1]:
import os
import pandas as pd
from langchain_matcher_async import process_all_batches_async
import nest_asyncio
from translator import translate_column_to_english
from local_matching import load_and_match, evaluate_matching, clean_file

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Define file paths
softwaregini_dir = "../softwaregini_files"
data_dir = "../data"
input_file_2 = os.path.join(softwaregini_dir, "organization_tools_2.csv")
ground_truth_file_2 = os.path.join(data_dir, "ground_truth_2.csv")

# Load and preprocess data
df_2_async = await translate_column_to_english(input_file_2)  # Assuming this is an async function
df_2_async = clean_file(df_2_async)  # Assuming this is a synchronous function

# Perform matching and evaluation
df_matched_2_async, potential_matches_2_async = load_and_match(df_2_async, similarity_cutoff=75)  # Assuming this is a synchronous function
results_2_async = evaluate_matching(df_matched_2_async, ground_truth_file_2, input_file_2)  # Assuming this is a synchronous function
print(f"Results before LLM for cutoff 75: ", results_2_async)

# Clean and prepare potential matches for LLM processing
potential_matches_2_async.columns = potential_matches_2_async.columns.str.strip()
potential_matches_2_async = potential_matches_2_async[['human_name', 'human_description']].rename(
    columns={'human_name': 'name', 'human_description': 'description'})

# Process potential matches asynchronously
df_potential_2_async , df_recommended_async = await process_all_batches_async(potential_matches_2_async, 95, 50)

# Merge results and evaluate
df_final_2_async = pd.concat([df_matched_2_async, df_recommended_async], axis=0)
df_final_2_async = clean_file(df_final_2_async)
results_2_async = evaluate_matching(df_final_2_async, ground_truth_file_2, input_file_2)
print(f"Results after LLM: ", results_2_async)

Translating 110 texts asynchronously...
Translation complete.
Cleaning file...
Computing embeddings for human descriptions...
Matching human descriptions to tools...
Amount of total Matches:  118
Amount of potential Matches:  3161
Number of dropped/deleted matches: 37
Number of final matches: 81
Number of potential matches: 144
Precision: 0.988
Recall: 0.851
F1-Score: 0.914
True Positives: 80
True Negatives: 15
False Positives: 1
False Negatives: 14
False Positives List:
False Negatives List:
Results before LLM for cutoff 75:  {'Precision': 0.9876543209876543, 'Recall': 0.851063829787234, 'F1-Score': 0.9142857142857143, 'True Positives': 80, 'True Negatives': 15, 'False Positives': 1, 'False Negatives': 14, 'False Positives List': [('linkedin', 'linkedin learning')], 'False Negatives List': [('sap fi/co', 'sap financial management'), ('sap sales and distribution', 'sap sales cloud'), ('sap leanix technology risk and compliance', 'leanix enterprise architecture management'), ('knowbe4',

100%|██████████| 3/3 [00:21<00:00,  7.26s/it]

Cleaning file...
Precision: 0.988
Recall: 0.862
F1-Score: 0.920
True Positives: 81
True Negatives: 15
False Positives: 1
False Negatives: 13
False Positives List:
False Negatives List:
Results after LLM:  {'Precision': 0.9878048780487805, 'Recall': 0.8617021276595744, 'F1-Score': 0.9204545454545454, 'True Positives': 81, 'True Negatives': 15, 'False Positives': 1, 'False Negatives': 13, 'False Positives List': [('linkedin', 'linkedin learning')], 'False Negatives List': [('sap fi/co', 'sap financial management'), ('sap sales and distribution', 'sap sales cloud'), ('sap leanix technology risk and compliance', 'leanix enterprise architecture management'), ('knowbe4', 'knowbe4 security awareness training'), ('linkedin', 'linkedin premium'), ('leanix enterprise architecture management', 'leanix enterprise architecture management'), ('sap leanix application portfolio management', 'leanix saas management platform'), ('bic enterprise risk', 'bic grc'), ('knowbe4', 'knowbe4 securitycoach'), ('

In [2]:
print(results_2_async)
print(df_recommended_async)

{'Precision': 0.9878048780487805, 'Recall': 0.8617021276595744, 'F1-Score': 0.9204545454545454, 'True Positives': 81, 'True Negatives': 15, 'False Positives': 1, 'False Negatives': 13, 'False Positives List': [('linkedin', 'linkedin learning')], 'False Negatives List': [('sap fi/co', 'sap financial management'), ('sap sales and distribution', 'sap sales cloud'), ('sap leanix technology risk and compliance', 'leanix enterprise architecture management'), ('knowbe4', 'knowbe4 security awareness training'), ('linkedin', 'linkedin premium'), ('leanix enterprise architecture management', 'leanix enterprise architecture management'), ('sap leanix application portfolio management', 'leanix saas management platform'), ('bic enterprise risk', 'bic grc'), ('knowbe4', 'knowbe4 securitycoach'), ('knowbe4', 'knowbe4 phisher'), ('sap leanix application portfolio management', 'leanix enterprise architecture management'), ('easy archive', 'easyarchive'), ('sap materials management', 'sap s/4hana cloud'